In [1]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import bs4
import pandas as pd
from itertools import repeat

In [2]:
# Path of WebDriver
driver = webdriver.Chrome('C:/Users/zheny/Downloads/chromedriver')

# Open Webpage Link
driver.get("https://www.linkedin.com")

# Locate username and password searchbox
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
                                                                       "input[name='session_key']")))

password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
                                                                       "input[name='session_password']")))

C:\Users\zheny\AppData\Local\Temp/ipykernel_18472/1264955643.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/zheny/Downloads/chromedriver')


In [3]:
# Enter username and passworod searchbox 
username.clear()
username.send_keys("zhenyuen_168_2009@hotmail.com") # fill in username here
password.clear()
password.send_keys("ngzhyuen5") # fill in password here
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [4]:
companies = ['American Express', 'Ebay', 'Visa', 'Amazon', 'Facebook', 'Exxon Mobil', 'General Electric', 'Home Depot', 'Pepsico', 'Pfizer', 'Procter and Gamble', 'United Health Group', 'Walmart']

In [5]:
def search_companies(x):
    SCROLL_PAUSE_TIME = 3
    driver.find_element(By.XPATH, "//input[contains(@aria-label, 'Search')]").click()
    driver.find_element(By.CLASS_NAME, 'search-global-typeahead__input').send_keys(x, Keys.ENTER)
    driver.implicitly_wait(10)
    driver.find_element(By.XPATH, '//button[text()="Companies"]').click()
    result = driver.find_elements(By.XPATH, "//li[@class = 'reusable-search__result-container ']")
    result[0].click()
    time.sleep(SCROLL_PAUSE_TIME)
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
    time.sleep(SCROLL_PAUSE_TIME)
    driver.find_element(By.XPATH, "//a[text() = 'Posts']").click()
    
    

In [6]:
def scrape_codes(company_name):
    driver.implicitly_wait(5)
    # Scroll down to bottom
    SCROLL_PAUSE_TIME = 1.5


    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    time.sleep(SCROLL_PAUSE_TIME)    
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
    
    postsbutton = driver.find_elements_by_xpath("//button[contains(@aria-label, 'see more')]")
    for x in range(0,len(postsbutton)):
        if postsbutton[x].is_displayed():
            postsbutton[x].click()
            
    # Get the page source
    source=driver.page_source
    test_soup=bs4.BeautifulSoup(source)

    article_list = test_soup.find_all("div" , class_="feed-shared-update-v2")
    text_list=[]
    date_list = []
    reaction_list = []
    comments_list = []
    repost_list = []

    for article in article_list:
        sentenceList=[]
        for i in (article.find_all("div", class_= "feed-shared-update-v2__description-wrapper")):
            sentenceList.append(i.text)
            
        text_list.append(" ".join(sentenceList))
        newTextList = [s.replace("\n", "") for s in text_list]
        
        # Get date
        date = article.find("div", class_ = "update-components-actor__meta relative").text
        date_list.append(date)
        
        # Get reaction
        reaction = article.find("span", class_='social-details-social-counts__reactions-count').text
        reaction_list.append(reaction)
        
        # Get comments
        try:
            comments = article.find("li", class_="social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--with-social-proof").text
            comments_list.append(comments)
        
        except AttributeError as e:
            comments_list.append(0)
        # repost = article.find("button", id = "ember8005").text
        # repost_list.append(repost)
        
    company_list=[]
        
    company_list.extend(repeat(company_name,len(newTextList)))
    company_df = pd.DataFrame(list(zip(newTextList, company_list, date_list, reaction_list, comments_list)),
                                                columns =['posts', 'company', 'date', 'reaction', 'comments'])
    return company_df

In [7]:
def get_parameters(x):
    import re 
    date = re.search('(\d+\s\w+\sago)', x['date'])
    comments = re.search('(\d+) comments', str(x['comments']))
    
    if date:
        x['revised_date'] = date[0]

    else:
        x['revised_date'] = ""
        
    if comments:
        x['revised_comments'] = comments[0].split()[0]
    elif x['comments']:
        x['revised_comments'] = 0
        
    else:
        x['revised_comments'] = 1
    return x

In [8]:
def get_days(x):
    value = 0
    if x == 'month' or x == 'months':
        value = 30
        
    elif x == 'week' or x == 'weeks':
        value = 7
        
    elif x == 'year':
        value = 365
        
    else:
        value = 1
        
    return value

In [9]:
from datetime import datetime, timedelta
def get_final_date(x):
    np_days = x['number'] * x['multiplier']
    date = datetime.today() - timedelta(days = np_days)
    x['date_final'] = date
    
    return x

In [14]:
count = 0
for i in companies:
    search_companies(i)
    df = scrape_codes(i)
    if count == 0:
        full_df = df
        
    else:
        full_df = pd.concat([full_df, df])

    count += 1  

In [16]:
full_df

,posts,company,date,reaction,comments
0,We celebrated our 2022 Customer Service Award...,American Express,"\n\n\nAmerican Express\n\n \n\n 1,7...","1,494",\n\n\n 23 comments\n \n\n
1,American Express Travel has unveiled its Tren...,American Express,"\n\n\nAmerican Express\n\n \n\n 1,7...",514,\n\n\n 4 comments\n \n\n
2,We flew our 2022 Customer Service Award winne...,American Express,"\n\n\nAmerican Express\n\n \n\n 1,7...","1,303",\n\n\n 19 comments\n \n\n
0,Today we announced our Q3 2022 results: Reven...,Ebay,"\n\n\neBay\n\n \n\n 521,203 followe...",344,\n\n\n 2 comments\n \n\n
1,Accessibility is a fundamental pillar of eBay...,Ebay,"\n\n\neBay\n\n \n\n 521,203 followe...",48,\n\n\n 2 comments\n \n\n
...,...,...,...,...,...
401,Walmart Chile will roll out electrical vehicl...,Walmart,"\n\n\nWalmart\n\n \n\n 4,126,506 fo...",875,\n\n\n 19 comments\n \n\n
402,"This Veterans Day, as in all those past, we’r...",Walmart,"\n\n\nWalmart\n\n \n\n 4,126,506 fo...",450,\n\n\n 13 comments\n \n\n
403,We believe we’re at our best when we promote ...,Walmart,"\n\n\nWalmart\n\n \n\n 4,126,506 fo...",610,\n\n\n 18 comments\n \n\n
404,We're proud to be named a 2021 TxN 20 Honoree...,Walmart,"\n\n\nWalmart\n\n \n\n 4,126,506 fo...",609,\n\n\n 35 comments\n \n\n


In [17]:
full_df.reset_index(drop = True, inplace = True)

In [18]:
full_df = full_df.apply(get_parameters, axis = 1)

In [19]:
full_df['number'] = full_df['revised_date'].apply(lambda x: x.split()[0])
full_df['type'] = full_df['revised_date'].apply(lambda x: x.split()[1])
full_df[['revised_comments', 'number']] = full_df[['revised_comments', 'number']].astype(int)


In [20]:
full_df['multiplier'] = full_df['type'].apply(get_days)
full_df = full_df.apply(get_final_date, axis = 1)
full_df['date_final'] = pd.to_datetime(full_df['date_final']).dt.date

In [21]:
full_df = full_df[['posts', 'company', 'date_final', 'revised_comments', 'reaction']]
full_df.columns = ['posts', 'company', 'date', 'comments', 'reaction']
full_df

,posts,company,date,comments,reaction
0,We celebrated our 2022 Customer Service Award...,American Express,2022-10-28,23,"1,494"
1,American Express Travel has unveiled its Tren...,American Express,2022-11-03,4,514
2,We flew our 2022 Customer Service Award winne...,American Express,2022-10-28,19,"1,303"
3,Today we announced our Q3 2022 results: Reven...,Ebay,2022-10-15,2,344
4,Accessibility is a fundamental pillar of eBay...,Ebay,2022-11-03,2,48
...,...,...,...,...,...
2876,Walmart Chile will roll out electrical vehicl...,Walmart,2021-11-04,19,875
2877,"This Veterans Day, as in all those past, we’r...",Walmart,2021-11-04,13,450
2878,We believe we’re at our best when we promote ...,Walmart,2021-11-04,18,610
2879,We're proud to be named a 2021 TxN 20 Honoree...,Walmart,2021-11-04,35,609


In [22]:
full_df['month_year'] = full_df['date'].apply(lambda x: x.strftime('%B-%Y')) 
full_df.sort_values('date', ascending = True, inplace = True)
full_df

,posts,company,date,comments,reaction,month_year
2880,We’re revolutionizing insulin access and affo...,Walmart,2021-11-04,12,461,November-2021
306,"T.R. Ramachandran, Group Country Manager for ...",Visa,2021-11-04,1,158,November-2021
305,Consumers are at the heart of everything Visa...,Visa,2021-11-04,3,263,November-2021
304,"What is Visa doing in the buy now, pay later ...",Visa,2021-11-04,1,183,November-2021
303,"Cuy Sheffield, Global head of Crypto, explain...",Visa,2021-11-04,0,213,November-2021
...,...,...,...,...,...,...
2336,"While attending the #IGNITION2022 Experience,...",United Health Group,2022-11-03,2,102,November-2022
1838,“The river has always taken care of the peopl...,Procter and Gamble,2022-11-03,3,293,November-2022
1839,#DYK that Black History Month is celebrated i...,Procter and Gamble,2022-11-03,1,57,November-2022
1690,It takes great bravery and honor to serve in ...,Pfizer,2022-11-03,2,171,November-2022


In [23]:
traditional = ['Exxon Mobil', 'General Electric', 'Home Depot', 'Pepsico', 'Pfizer', 'Procter and Gamble', 'United Health Group', 'Walmart']
innovative = ['American Express', 'Ebay', 'Visa', 'Amazon', 'Facebook']

In [24]:
full_df.loc[full_df['company'].isin(traditional), 'type'] = 'traditional'
full_df.loc[full_df['company'].isin(innovative), 'type'] = 'innovative'

In [25]:
full_df.loc[full_df['type'].isna()]['company'].unique()

array([], dtype=object)

In [26]:
full_df.reset_index(drop = True)

,posts,company,date,comments,reaction,month_year,type
0,We’re revolutionizing insulin access and affo...,Walmart,2021-11-04,12,461,November-2021,traditional
1,"T.R. Ramachandran, Group Country Manager for ...",Visa,2021-11-04,1,158,November-2021,innovative
2,Consumers are at the heart of everything Visa...,Visa,2021-11-04,3,263,November-2021,innovative
3,"What is Visa doing in the buy now, pay later ...",Visa,2021-11-04,1,183,November-2021,innovative
4,"Cuy Sheffield, Global head of Crypto, explain...",Visa,2021-11-04,0,213,November-2021,innovative
...,...,...,...,...,...,...,...
2876,"While attending the #IGNITION2022 Experience,...",United Health Group,2022-11-03,2,102,November-2022,traditional
2877,“The river has always taken care of the peopl...,Procter and Gamble,2022-11-03,3,293,November-2022,traditional
2878,#DYK that Black History Month is celebrated i...,Procter and Gamble,2022-11-03,1,57,November-2022,traditional
2879,It takes great bravery and honor to serve in ...,Pfizer,2022-11-03,2,171,November-2022,traditional


In [27]:
full_df.to_csv(r'C:\Users\zheny\Downloads\webscrape_companies.csv', index = False)

### Visualise Data

In [14]:
full_df['month'] = pd.to_datetime(full_df['date']).dt.month
full_df['year'] = pd.to_datetime(full_df['date']).dt.year

In [22]:
count_df = full_df.groupby(['month', 'year', 'type', 'month_year'])['posts'].count().reset_index(name = 'count').sort_values(['year', 'month'])
count_df = count_df.loc[count_df['month_year'] != 'November-2022']

In [23]:
import plotly.express as px
fig = px.bar(count_df, x='month_year', y='count',
             color='type', height=400, text_auto=True) 

fig.show()

In [34]:
company_count = full_df['company'].value_counts().reset_index()
company_count.columns = ['company', 'count']
company_count = pd.merge(company_count, full_df[['company', 'type']], on = 'company', how = 'left').drop_duplicates()
company_count

,company,count,type
0,Procter and Gamble,500,traditional
500,Amazon,474,innovative
974,Walmart,406,traditional
1380,General Electric,339,traditional
1719,Facebook,244,innovative
1963,Visa,223,innovative
2186,Pfizer,146,traditional
2332,United Health Group,139,traditional
2471,Pepsico,138,traditional
2609,Exxon Mobil,100,traditional


In [55]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
            x=company_count['count'],
            y=company_count['company'],
            text = company_count['count'],
            orientation='h'))
fig.update_layout(
            title= '<b>Company Breakdown by post</b>',
            font=dict(
                size=12,
                color="Black"
            )
        )
fig.show()